# Crypto Convert System - Improvements & Best Practices

This notebook outlines comprehensive improvements for the crypto-convert-system project, covering architecture, code organization, performance, security, and more.

## Estrutura e Organização do Código

Uma arquitetura modular bem definida é essencial para a manutenção e escalabilidade do projeto. Vamos explorar como implementar o padrão MVC (Model-View-Controller) para melhorar a organização do código.

In [ ]:
# Exemplo de implementação do padrão MVC

# Model - Lida com a lógica de negócios e dados
class CryptoModel:
    def __init__(self):
        self.rates = {}
        self.base_currency = "USD"
    
    def fetch_rates(self, api_key):
        """Busca taxas de câmbio da API"""
        import requests
        url = f"https://api.coincap.io/v2/rates"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Processar dados
            for item in data.get('data', []):
                self.rates[item['symbol']] = float(item['rateUsd'])
            return True
        return False
    
    def convert(self, amount, from_currency, to_currency):
        """Converte valor entre moedas"""
        if from_currency not in self.rates or to_currency not in self.rates:
            raise ValueError("Moeda não suportada")
        
        # Conversão para USD e depois para moeda alvo
        amount_in_usd = amount / self.rates[from_currency]
        return amount_in_usd * self.rates[to_currency]

# Controller - Intermedia a comunicação entre Model e View
class CryptoController:
    def __init__(self, model):
        self.model = model
    
    def initialize(self, api_key):
        """Inicializa o sistema"""
        return self.model.fetch_rates(api_key)
    
    def perform_conversion(self, amount, from_currency, to_currency):
        """Realiza a conversão"""
        try:
            return self.model.convert(amount, from_currency, to_currency)
        except ValueError as e:
            return {"error": str(e)}

# View - Interface com o usuário (simplificada para demonstração)
class CryptoView:
    def __init__(self, controller):
        self.controller = controller
    
    def display_result(self, result):
        """Exibe o resultado da conversão"""
        if isinstance(result, dict) and "error" in result:
            print(f"Erro: {result['error']}")
        else:
            print(f"Resultado da conversão: {result:.2f}")
    
    # Em um aplicativo real, aqui teríamos métodos para renderizar UI

### Benefícios da Arquitetura MVC

1. **Separação de Responsabilidades**: Cada componente tem uma função específica
2. **Manutenção Simplificada**: Alterações em um componente não afetam os outros
3. **Testabilidade**: Facilita os testes unitários
4. **Reutilização**: Componentes podem ser reutilizados em diferentes partes do sistema

### Estrutura de Diretórios Recomendada

In [ ]:
# Exemplo de estrutura de diretórios para o projeto
from pathlib import Path

# Função para visualizar a estrutura de diretórios
def print_directory_structure(base_path, indent=0):
    print("  " * indent + str(base_path.name) + "/")
    for path in sorted(base_path.glob("*")):
        if path.is_dir():
            print_directory_structure(path, indent + 1)
        else:
            print("  " * (indent + 1) + str(path.name))

# Estrutura recomendada
project_structure = {
    "crypto-convert-system": [
        "src", [
            "models", ["currencyModel.js", "userModel.js"],
            "views", ["converterView.js", "historyView.js"],
            "controllers", ["appController.js", "converterController.js"],
            "utils", ["apiClient.js", "validation.js", "formatters.js"],
            "services", ["authService.js", "storageService.js"],
            "assets", ["css", "images", "fonts"]
        ],
        "tests", ["unit", "integration", "e2e"],
        "docs", ["api.md", "setup.md"],
        "config", ["webpack.config.js", "babel.config.js"],
        "package.json",
        "README.md"
    ]
}

# Visualização da estrutura (apenas demonstrativo)
print("Estrutura de Diretórios Recomendada:")
print("crypto-convert-system/")
print("  ├── src/")
print("  │   ├── models/")
print("  │   │   ├── currencyModel.js")
print("  │   │   └── userModel.js")
print("  │   ├── views/")
print("  │   │   ├── converterView.js")
print("  │   │   └── historyView.js")
print("  │   ├── controllers/")
print("  │   │   ├── appController.js")
print("  │   │   └── converterController.js")
print("  │   ├── utils/")
print("  │   │   ├── apiClient.js")
print("  │   │   ├── validation.js")
print("  │   │   └── formatters.js")
print("  │   ├── services/")
print("  │   │   ├── authService.js")
print("  │   │   └── storageService.js")
print("  │   └── assets/")
print("  │       ├── css/")
print("  │       ├── images/")
print("  │       └── fonts/")
print("  ├── tests/")
print("  │   ├── unit/")
print("  │   ├── integration/")
print("  │   └── e2e/")
print("  ├── docs/")
print("  │   ├── api.md")
print("  │   └── setup.md")
print("  ├── config/")
print("  │   ├── webpack.config.js")
print("  │   └── babel.config.js")
print("  ├── package.json")
print("  └── README.md")

## Gerenciamento de Estado e Variáveis

O gerenciamento eficiente de estado é crucial para aplicações modernas. Vamos explorar como implementar um sistema de gerenciamento de estado simples, limitar o escopo de variáveis e utilizar módulos ES6.

In [ ]:
// Implementação simples de um gerenciador de estado
const createStore = (initialState = {}) => {
  let state = { ...initialState };
  const listeners = new Set();

  const getState = () => ({ ...state });

  const setState = (newState) => {
    state = { ...state, ...newState };
    notifyListeners();
  };

  const subscribe = (listener) => {
    listeners.add(listener);
    return () => listeners.delete(listener);
  };

  const notifyListeners = () => {
    listeners.forEach(listener => listener(state));
  };

  return { getState, setState, subscribe };
};

// Exemplo de uso
const appStore = createStore({
  currencies: [],
  selectedCurrency: 'BTC',
  conversionRate: null,
  loading: false,
  error: null
});

// Subscriber que reage às mudanças de estado
const unsubscribe = appStore.subscribe((state) => {
  console.log('Estado atualizado:', state);
  // Aqui você atualizaria a UI com os novos dados
});

// Ação que altera o estado
const fetchCurrencies = async () => {
  appStore.setState({ loading: true, error: null });
  
  try {
    // Simulação de chamada de API
    const response = await fetch('https://api.coincap.io/v2/rates');
    const data = await response.json();
    
    appStore.setState({ 
      currencies: data.data,
      loading: false 
    });
  } catch (error) {
    appStore.setState({ 
      error: 'Falha ao carregar moedas',
      loading: false 
    });
  }
};

// Para demonstração - não executa
console.log('Exemplo de implementação de gerenciamento de estado');

### Módulos ES6 para Melhor Organização

Utilizar módulos ES6 melhora a organização do código, limita o escopo de variáveis e facilita a manutenção. 
Veja um exemplo de como estruturar os arquivos usando o sistema de módulos:

In [ ]:
// api.js - Módulo de serviços de API
export const fetchRates = async () => {
  try {
    const response = await fetch('https://api.coincap.io/v2/rates');
    if (!response.ok) throw new Error('Network response was not ok');
    return await response.json();
  } catch (error) {
    console.error('Error fetching rates:', error);
    throw error;
  }
};

export const fetchCryptoDetails = async (cryptoId) => {
  try {
    const response = await fetch(`https://api.coincap.io/v2/assets/${cryptoId}`);
    if (!response.ok) throw new Error('Network response was not ok');
    return await response.json();
  } catch (error) {
    console.error(`Error fetching details for ${cryptoId}:`, error);
    throw error;
  }
};

// converter.js - Módulo de conversão
export const convertCurrency = (amount, fromRate, toRate) => {
  if (typeof amount !== 'number' || amount < 0) {
    throw new TypeError('Amount must be a positive number');
  }
  
  if (typeof fromRate !== 'number' || fromRate <= 0) {
    throw new TypeError('From rate must be a positive number');
  }
  
  if (typeof toRate !== 'number' || toRate <= 0) {
    throw new TypeError('To rate must be a positive number');
  }
  
  // Conversão para USD e depois para moeda de destino
  const amountInUSD = amount / fromRate;
  return amountInUSD * toRate;
};

// formatter.js - Módulo de formatação
export const formatCurrency = (amount, currency = 'USD') => {
  return new Intl.NumberFormat('en-US', {
    style: 'currency',
    currency: currency,
    minimumFractionDigits: 2,
    maximumFractionDigits: 6
  }).format(amount);
};

// Exemplo de uso em app.js
/*
import { fetchRates } from './api.js';
import { convertCurrency } from './converter.js';
import { formatCurrency } from './formatter.js';

// Uso dos módulos
async function performConversion(amount, fromCurrency, toCurrency) {
  try {
    const { data } = await fetchRates();
    
    const fromRate = data.find(c => c.symbol === fromCurrency).rateUsd;
    const toRate = data.find(c => c.symbol === toCurrency).rateUsd;
    
    const result = convertCurrency(amount, fromRate, toRate);
    return formatCurrency(result, toCurrency);
  } catch (error) {
    console.error('Conversion error:', error);
    throw error;
  }
}
*/

### Boas Práticas para Gerenciamento de Variáveis

1. **Use const por padrão** - Utilize `const` para declarar variáveis que não serão reatribuídas
2. **Prefira let sobre var** - `let` tem escopo de bloco, o que reduz bugs
3. **Minimize variáveis globais** - Encapsule em módulos ou funções
4. **Nomeie variáveis de forma descritiva** - Nomes claros e significativos 
5. **Desestruturação** - Use para extrair dados de objetos e arrays

## Tratamento de Erros

Um sistema eficiente de tratamento de erros é essencial para a robustez da aplicação. Vamos implementar estratégias para captura, log e exibição de erros.

In [ ]:
// Sistema de logging customizado
const LogLevel = {
  ERROR: 0,
  WARN: 1,
  INFO: 2,
  DEBUG: 3
};

class Logger {
  constructor(moduleName, logLevel = LogLevel.INFO) {
    this.moduleName = moduleName;
    this.logLevel = logLevel;
  }

  formatMessage(level, message) {
    const timestamp = new Date().toISOString();
    return `[${timestamp}] [${level}] [${this.moduleName}]: ${message}`;
  }

  error(message, error) {
    if (this.logLevel >= LogLevel.ERROR) {
      console.error(this.formatMessage('ERROR', message));
      if (error && error.stack) {
        console.error(error.stack);
      }
      
      // Em produção, você pode enviar erros para um serviço como Sentry
      // this.sendToErrorService(message, error);
    }
  }

  warn(message) {
    if (this.logLevel >= LogLevel.WARN) {
      console.warn(this.formatMessage('WARN', message));
    }
  }

  info(message) {
    if (this.logLevel >= LogLevel.INFO) {
      console.info(this.formatMessage('INFO', message));
    }
  }

  debug(message) {
    if (this.logLevel >= LogLevel.DEBUG) {
      console.debug(this.formatMessage('DEBUG', message));
    }
  }

  // sendToErrorService(message, error) {
  //   // Integração com serviços como Sentry, LogRocket, etc.
  // }
}

// Exemplo de classe de erros customizados
class AppError extends Error {
  constructor(message, code, httpStatus = 400) {
    super(message);
    this.name = this.constructor.name;
    this.code = code;
    this.httpStatus = httpStatus;
    
    // Preserva o stack trace
    if (Error.captureStackTrace) {
      Error.captureStackTrace(this, this.constructor);
    }
  }
}

class ValidationError extends AppError {
  constructor(message, field) {
    super(message, 'VALIDATION_ERROR', 400);
    this.field = field;
  }
}

class ApiError extends AppError {
  constructor(message, apiResponse) {
    super(message, 'API_ERROR', apiResponse?.status || 500);
    this.apiResponse = apiResponse;
  }
}

// Exemplo de uso
const logger = new Logger('CryptoConverter', LogLevel.DEBUG);

async function fetchCryptoPrices() {
  logger.debug('Iniciando busca de preços de criptomoedas');
  
  try {
    const response = await fetch('https://api.coincap.io/v2/rates');
    
    if (!response.ok) {
      throw new ApiError('Falha ao buscar preços', response);
    }
    
    const data = await response.json();
    logger.info(`Recebidos dados de ${data.data.length} moedas`);
    return data;
    
  } catch (error) {
    if (error instanceof AppError) {
      logger.error(`Erro na aplicação: ${error.message}`, error);
    } else if (error.name === 'TypeError') {
      logger.error('Erro de rede - verifique sua conexão', error);
    } else {
      logger.error('Erro desconhecido', error);
    }
    
    // Propague o erro para ser tratado em nível superior
    throw error;
  }
}

// Validação com tratamento de erros
function validateConversionInput(amount, fromCurrency, toCurrency) {
  if (!amount || isNaN(parseFloat(amount))) {
    throw new ValidationError('O valor deve ser um número válido', 'amount');
  }
  
  if (parseFloat(amount) <= 0) {
    throw new ValidationError('O valor deve ser maior que zero', 'amount');
  }
  
  if (!fromCurrency) {
    throw new ValidationError('Moeda de origem é obrigatória', 'fromCurrency');
  }
  
  if (!toCurrency) {
    throw new ValidationError('Moeda de destino é obrigatória', 'toCurrency');
  }
  
  if (fromCurrency === toCurrency) {
    throw new ValidationError('As moedas de origem e destino devem ser diferentes', 'toCurrency');
  }
}

// Manipulador global de erros (para uma aplicação web)
function setupGlobalErrorHandler() {
  window.addEventListener('error', (event) => {
    logger.error('Erro não tratado capturado:', event.error);
    // Exibir mensagem amigável para o usuário
    showUserFriendlyError('Ocorreu um erro inesperado. Tente novamente mais tarde.');
    return false; // Permite que o erro seja propagado
  });
  
  window.addEventListener('unhandledrejection', (event) => {
    logger.error('Promise rejeitada não tratada:', event.reason);
    showUserFriendlyError('Falha na operação. Verifique sua conexão e tente novamente.');
    return false;
  });
}

// Função para exibir erro para o usuário
function showUserFriendlyError(message) {
  // Em uma aplicação real, isso exibiria uma notificação na UI
  console.log(`%c[Erro para o usuário] ${message}`, 'color: white; background-color: red; padding: 2px 5px; border-radius: 3px;');
}

// Exemplo de como seria o tratamento no nível da aplicação
async function handleConversion(amount, fromCurrency, toCurrency) {
  try {
    validateConversionInput(amount, fromCurrency, toCurrency);
    
    const result = await performConversion(amount, fromCurrency, toCurrency);
    return result;
    
  } catch (error) {
    if (error instanceof ValidationError) {
      showUserFriendlyError(`Erro de validação: ${error.message}`);
    } else if (error instanceof ApiError) {
      showUserFriendlyError('Não foi possível obter os dados de conversão. Tente novamente mais tarde.');
    } else {
      showUserFriendlyError('Ocorreu um erro durante a conversão.');
    }
    
    logger.error('Erro durante a conversão:', error);
    return null;
  }
}

// Para demonstração
console.log('Exemplo de sistema de tratamento de erros implementado');

### Melhores Práticas para Tratamento de Erros

1. **Crie uma hierarquia de erros** - Erros específicos ajudam a identificar problemas
2. **Centralize o logging** - Sistema unificado facilita o diagnóstico
3. **Seja específico nas mensagens** - Mensagens claras facilitam a depuração
4. **Separe erros técnicos de mensagens para usuários** - Usuários não precisam ver stacktraces
5. **Use try/catch em código assíncrono** - Capture erros em promessas e funções async/await
6. **Implemente captura global de erros** - Para tratar erros não capturados
7. **Adicione contexto aos erros** - Informações adicionais para facilitar a depuração

## Performance e Otimização

A otimização de performance é crucial para uma boa experiência do usuário. Vamos explorar algumas técnicas de otimização.

In [ ]:
// Implementação de função com memoização (cache)
function memoize(fn) {
  const cache = new Map();
  
  return function(...args) {
    const key = JSON.stringify(args);
    
    if (cache.has(key)) {
      console.log('Cache hit!');
      return cache.get(key);
    }
    
    console.log('Cache miss!');
    const result = fn.apply(this, args);
    cache.set(key, result);
    return result;
  };
}

// Função que seria beneficiada por memoização
const calculateCompoundInterest = (principal, rate, time, compounds) => {
  // Simulando operação complexa
  console.log('Performing expensive calculation...');
  return principal * Math.pow(1 + rate / compounds, compounds * time);
};

// Versão memoizada
const memoizedCalculateInterest = memoize(calculateCompoundInterest);

// Exemplo de uso
console.log(memoizedCalculateInterest(1000, 0.05, 5, 12)); // Cache miss
console.log(memoizedCalculateInterest(1000, 0.05, 5, 12)); // Cache hit
console.log(memoizedCalculateInterest(2000, 0.05, 5, 12)); // Cache miss

// Implementação de debounce para eventos frequentes
function debounce(func, wait) {
  let timeout;
  
  return function executedFunction(...args) {
    const later = () => {
      clearTimeout(timeout);
      func(...args);
    };
    
    clearTimeout(timeout);
    timeout = setTimeout(later, wait);
  };
}

// Exemplo de uso de debounce
const handleSearch = (query) => {
  console.log(`Searching for: ${query}`);
  // Lógica de busca
};

// Versão com debounce - só executa após 300ms de inatividade
const debouncedSearch = debounce(handleSearch, 300);

// Simulação de digitação rápida do usuário
// debouncedSearch('c');
// debouncedSearch('cr');
// debouncedSearch('cry');
// debouncedSearch('cryp'); // Apenas esta chamada seria executada

// Implementação de throttle para limitar a frequência de execução
function throttle(func, limit) {
  let inThrottle;
  
  return function(...args) {
    if (!inThrottle) {
      func.apply(this, args);
      inThrottle = true;
      setTimeout(() => inThrottle = false, limit);
    }
  };
}

// Exemplo de uso de throttle
const handleScroll = () => {
  console.log('Scroll event processed');
  // Lógica de processamento do scroll
};

// Versão com throttle - executa no máximo uma vez a cada 100ms
const throttledScroll = throttle(handleScroll, 100);

// Em uma aplicação real:
// window.addEventListener('scroll', throttledScroll);

// Exemplo de API para observar a performance
const measurePerformance = (label, callback) => {
  if (performance && performance.mark) {
    performance.mark(`${label}-start`);
    
    const result = callback();
    
    performance.mark(`${label}-end`);
    performance.measure(label, `${label}-start`, `${label}-end`);
    
    const measures = performance.getEntriesByName(label);
    console.log(`${label} took ${measures[0].duration.toFixed(2)}ms`);
    
    return result;
  } else {
    return callback();
  }
};

// Exemplo de uso
const result = measurePerformance('sorting-algorithm', () => {
  // Simulando operação que consome tempo
  const arr = Array.from({length: 100000}, () => Math.random());
  return arr.sort((a, b) => a - b);
});

### Otimização do Uso de API

Uma das principais fontes de gargalos em aplicações web é o uso de APIs externas. Vamos implementar estratégias de otimização:

In [ ]:
// Classe para gerenciar chamadas à API com cache
class ApiClient {
  constructor(baseUrl, cacheTime = 5 * 60 * 1000) { // 5 minutos padrão
    this.baseUrl = baseUrl;
    this.cacheTime = cacheTime;
    this.cache = new Map();
  }
  
  async get(endpoint, params = {}, forceRefresh = false) {
    const url = this._buildUrl(endpoint, params);
    const cacheKey = url;
    
    // Verificar cache se não forçar atualização
    if (!forceRefresh && this.cache.has(cacheKey)) {
      const cachedData = this.cache.get(cacheKey);
      if (Date.now() < cachedData.expiry) {
        console.log(`Cache hit for ${url}`);
        return cachedData.data;
      }
      // Cache expirado, remove
      this.cache.delete(cacheKey);
    }
    
    try {
      console.log(`Fetching from API: ${url}`);
      const response = await fetch(url);
      
      if (!response.ok) {
        throw new Error(`API error: ${response.status} ${response.statusText}`);
      }
      
      const data = await response.json();
      
      // Armazenar no cache
      this.cache.set(cacheKey, {
        data,
        expiry: Date.now() + this.cacheTime
      });
      
      return data;
    } catch (error) {
      console.error(`Error fetching ${url}:`, error);
      throw error;
    }
  }
  
  _buildUrl(endpoint, params) {
    const url = new URL(`${this.baseUrl}${endpoint}`);
    
    // Adicionar parâmetros de query
    Object.entries(params).forEach(([key, value]) => {
      if (value !== undefined && value !== null) {
        url.searchParams.append(key, value);
      }
    });
    
    return url.toString();
  }
  
  clearCache() {
    this.cache.clear();
  }
  
  invalidateCache(endpoint) {
    // Remove entradas de cache que começam com o endpoint
    for (const key of this.cache.keys()) {
      if (key.includes(`${this.baseUrl}${endpoint}`)) {
        this.cache.delete(key);
      }
    }
  }
}

// Exemplo de uso da classe ApiClient
const cryptoApi = new ApiClient('https://api.coincap.io/v2/');

async function demonstrateApiCaching() {
  try {
    console.log('Primeira chamada - sem cache:');
    const firstCall = await cryptoApi.get('rates');
    console.log(`Recebidos ${firstCall.data.length} registros`);
    
    console.log('\nSegunda chamada - com cache:');
    const secondCall = await cryptoApi.get('rates');
    console.log(`Recebidos ${secondCall.data.length} registros do cache`);
    
    console.log('\nTerceira chamada - forçando atualização:');
    const thirdCall = await cryptoApi.get('rates', {}, true);
    console.log(`Recebidos ${thirdCall.data.length} registros atualizados`);
    
  } catch (error) {
    console.error('Erro na demonstração:', error);
  }
}

// Para demonstração
console.log('Demonstração de técnicas de otimização de performance');
// Não executa realmente a função para evitar chamadas API
// demonstrateApiCaching();

### Técnicas Adicionais de Otimização

1. **Lazy Loading de Componentes** - Carregue componentes apenas quando necessário
2. **Virtualização de Listas** - Renderize apenas os itens visíveis em listas longas
3. **Compressão de Recursos** - Minifique CSS/JS e comprima imagens
4. **Uso de Service Workers** - Para caching de recursos e funcionalidade offline
5. **Web Workers** - Realize cálculos intensivos em threads separadas
6. **Otimização do Critical Rendering Path** - Priorize carregamento de recursos críticos
7. **Paginação e Carregamento Infinito** - Carregue dados em lotes menores

## Segurança

A segurança é um aspecto crucial de qualquer aplicação. Vamos explorar técnicas para melhorar a segurança do sistema de conversão de criptomoedas.

In [ ]:
// Sanitização de entrada de usuário
class InputSanitizer {
  static sanitizeString(input) {
    if (typeof input !== 'string') return '';
    
    // Remove HTML tags e scripts
    const sanitized = input
      .replace(/</g, '&lt;')
      .replace(/>/g, '&gt;')
      .replace(/&/g, '&amp;')
      .replace(/"/g, '&quot;')
      .replace(/'/g, '&#x27;')
      .replace(/\//g, '&#x2F;');
    
    return sanitized.trim();
  }
  
  static sanitizeNumber(input) {
    // Converte para string e remove caracteres não numéricos
    const cleaned = String(input).replace(/[^0-9.-]/g, '');
    const num = parseFloat(cleaned);
    return isNaN(num) ? 0 : num;
  }
  
  static sanitizeBoolean(input) {
    if (typeof input === 'boolean') return input;
    if (typeof input === 'string') {
      return ['true', '1', 'yes', 'y'].includes(input.toLowerCase());
    }
    return Boolean(input);
  }
  
  static sanitizeCurrency(input) {
    // Assume que códigos de moeda são alfanuméricos e maiúsculos
    if (typeof input !== 'string') return '';
    
    // Remove caracteres não alfanuméricos e converte para maiúsculas
    return input.replace(/[^a-zA-Z0-9]/g, '').toUpperCase();
  }
}

// Validação de dados de entrada
class InputValidator {
  static validateAmount(amount) {
    const num = parseFloat(amount);
    
    if (isNaN(num)) {
      return { valid: false, message: 'O valor deve ser um número' };
    }
    
    if (num <= 0) {
      return { valid: false, message: 'O valor deve ser maior que zero' };
    }
    
    if (num > 1000000000) { // 1 bilhão como limite
      return { valid: false, message: 'O valor excede o limite máximo' };
    }
    
    return { valid: true };
  }
  
  static validateCurrency(currency, availableCurrencies) {
    if (!currency) {
      return { valid: false, message: 'Moeda é obrigatória' };
    }
    
    if (availableCurrencies && !availableCurrencies.includes(currency)) {
      return { valid: false, message: 'Moeda não suportada' };
    }
    
    return { valid: true };
  }
  
  static validateApiKey(apiKey) {
    if (!apiKey || typeof apiKey !== 'string') {
      return { valid: false, message: 'API key inválida' };
    }
    
    // Verifica se a API key tem o formato esperado
    // Por exemplo, uma chave hexadecimal de 32 caracteres
    if (!/^[a-f0-9]{32}$/i.test(apiKey)) {
      return { valid: false, message: 'Formato de API key inválido' };
    }
    
    return { valid: true };
  }
}

// Exemplo de proteção contra ataques CSRF
class CSRFProtection {
  static generateToken() {
    // Em uma implementação real, usaríamos uma biblioteca de criptografia
    // como crypto-js para gerar um token seguro
    const randomBytes = new Uint8Array(16);
    crypto.getRandomValues(randomBytes);
    return Array.from(randomBytes)
      .map(b => b.toString(16).padStart(2, '0'))
      .join('');
  }
  
  static setTokenCookie(token) {
    document.cookie = `csrfToken=${token}; path=/; SameSite=Strict; Secure`;
  }
  
  static getTokenFromCookie() {
    const match = document.cookie.match(/csrfToken=([^;]+)/);
    return match ? match[1] : null;
  }
  
  static addTokenToForm(formElement) {
    const token = this.getTokenFromCookie();
    if (!token) return;
    
    let input = formElement.querySelector('input[name="csrf_token"]');
    if (!input) {
      input = document.createElement('input');
      input.type = 'hidden';
      input.name = 'csrf_token';
      formElement.appendChild(input);
    }
    
    input.value = token;
  }
  
  static verifyToken(requestToken) {
    const storedToken = this.getTokenFromCookie();
    return storedToken && storedToken === requestToken;
  }
}

// Proteção de dados sensíveis
class SensitiveDataHandler {
  constructor() {
    this.encryptionKey = null;
  }
  
  async initialize() {
    // Em uma aplicação real, a chave poderia vir de um serviço seguro
    // ou ser derivada de credenciais do usuário
    // Aqui estamos apenas gerando uma para demonstração
    const keyMaterial = await crypto.subtle.generateKey(
      { name: 'AES-GCM', length: 256 },
      true,
      ['encrypt', 'decrypt']
    );
    
    this.encryptionKey = keyMaterial;
    return true;
  }
  
  async encrypt(data) {
    if (!this.encryptionKey) {
      throw new Error('Encryption not initialized');
    }
    
    // Converter dados para formato adequado
    const encoder = new TextEncoder();
    const dataBuffer = encoder.encode(JSON.stringify(data));
    
    // Gerar IV (Initialization Vector)
    const iv = crypto.getRandomValues(new Uint8Array(12));
    
    // Encriptar
    const encryptedData = await crypto.subtle.encrypt(
      { name: 'AES-GCM', iv },
      this.encryptionKey,
      dataBuffer
    );
    
    // Converter para formato armazenável
    const encryptedArray = Array.from(new Uint8Array(encryptedData));
    const ivArray = Array.from(iv);
    
    return {
      encryptedData: encryptedArray,
      iv: ivArray
    };
  }
  
  async decrypt(encryptedObject) {
    if (!this.encryptionKey) {
      throw new Error('Encryption not initialized');
    }
    
    const { encryptedData, iv } = encryptedObject;
    
    // Converter de volta para ArrayBuffer
    const encryptedBuffer = new Uint8Array(encryptedData).buffer;
    const ivBuffer = new Uint8Array(iv);
    
    // Decriptar
    const decryptedBuffer = await crypto.subtle.decrypt(
      { name: 'AES-GCM', iv: ivBuffer },
      this.encryptionKey,
      encryptedBuffer
    );
    
    // Converter para string
    const decoder = new TextDecoder();
    const decryptedString = decoder.decode(decryptedBuffer);
    
    return JSON.parse(decryptedString);
  }
  
  maskApiKey(apiKey) {
    if (!apiKey || typeof apiKey !== 'string') return '';
    
    // Mostra apenas os primeiros e últimos 4 caracteres
    const length = apiKey.length;
    if (length <= 8) return '********';
    
    return apiKey.substring(0, 4) + 
           '*'.repeat(length - 8) + 
           apiKey.substring(length - 4);
  }
}

// Exemplo de uso
console.log('Sanitização:', InputSanitizer.sanitizeString('<script>alert("XSS")</script>'));
console.log('Validação de valor:', InputValidator.validateAmount('123.45'));
console.log('Validação de valor inválido:', InputValidator.validateAmount('-10'));
console.log('CSRF Token:', CSRFProtection.generateToken());

// Demonstração de mascaramento de API key
const sensitiveHandler = new SensitiveDataHandler();
console.log('API Key mascarada:', sensitiveHandler.maskApiKey('abcd1234efgh5678ijkl9012'));

### Melhores Práticas de Segurança

1. **Validação de Entrada** - Sempre valide e sanitize dados de entrada
2. **HTTPS** - Use conexões seguras para todas as comunicações
3. **Content Security Policy (CSP)** - Restrinja fontes de conteúdo para prevenir XSS
4. **Não exponha informações sensíveis** - Mantenha chaves de API no lado do servidor
5. **Rate Limiting** - Implemente limites de requisições para prevenir abusos
6. **Atualize Dependências** - Mantenha bibliotecas atualizadas para corrigir vulnerabilidades
7. **Armazenamento Seguro** - Use armazenamento seguro para dados sensíveis
8. **Autenticação e Autorização** - Implemente um sistema robusto para controle de acesso

## Testabilidade

Escrever código testável é essencial para a manutenção a longo prazo. Vamos explorar algumas técnicas para melhorar a testabilidade do projeto.

In [ ]:
// Exemplo de código refatorado para maior testabilidade
// Usando injeção de dependências

// Antes: Difícil de testar
/*
class CryptoConverter {
  async convert(amount, fromCurrency, toCurrency) {
    // Dependência direta da API externa
    const response = await fetch('https://api.coincap.io/v2/rates');
    const data = await response.json();
    
    const fromRate = data.data.find(c => c.symbol === fromCurrency).rateUsd;
    const toRate = data.data.find(c => c.symbol === toCurrency).rateUsd;
    
    return (amount / fromRate) * toRate;
  }
}
*/

// Depois: Testável através de injeção de dependências
class CryptoConverter {
  constructor(apiClient) {
    this.apiClient = apiClient;
  }
  
  async convert(amount, fromCurrency, toCurrency) {
    // Usa a dependência injetada
    const data = await this.apiClient.getRates();
    
    const fromRate = this.findRate(data, fromCurrency);
    const toRate = this.findRate(data, toCurrency);
    
    if (!fromRate || !toRate) {
      throw new Error('Currency not found');
    }
    
    return (amount / fromRate) * toRate;
  }
  
  findRate(data, currency) {
    const currencyData = data.find(c => c.symbol === currency);
    return currencyData ? parseFloat(currencyData.rateUsd) : null;
  }
}

// Interface da API que pode ser implementada por um mock para testes
class CryptoApiClient {
  async getRates() {
    const response = await fetch('https://api.coincap.io/v2/rates');
    const data = await response.json();
    return data.data;
  }
}

// Para testes, podemos criar um mock desta API
class MockCryptoApiClient {
  constructor(mockData) {
    this.mockData = mockData || [
      { symbol: 'USD', rateUsd: '1' },
      { symbol: 'BTC', rateUsd: '50000' },
      { symbol: 'ETH', rateUsd: '3000' }
    ];
  }
  
  async getRates() {
    // Simula delay de rede
    await new Promise(resolve => setTimeout(resolve, 10));
    return this.mockData;
  }
}

// Exemplo de teste unitário (usando sintaxe similar a Jest)
function describeTest(description, fn) {
  console.log(`\n--- ${description} ---`);
  fn();
}

function it(description, fn) {
  console.log(`\nTest: ${description}`);
  try {
    fn();
    console.log('✅ Test passed');
  } catch (error) {
    console.log(`❌ Test failed: ${error.message}`);
  }
}

function expect(actual) {
  return {
    toBe(expected) {
      if (actual !== expected) {
        throw new Error(`Expected ${expected} but got ${actual}`);
      }
    },
    toBeCloseTo(expected, precision = 2) {
      const factor = Math.pow(10, precision);
      const roundedActual = Math.round(actual * factor) / factor;
      const roundedExpected = Math.round(expected * factor) / factor;
      
      if (roundedActual !== roundedExpected) {
        throw new Error(`Expected ${expected} to be close to ${actual}`);
      }
    },
    toThrow() {
      if (!(actual instanceof Function)) {
        throw new Error('Expected a function');
      }
      
      try {
        actual();
        throw new Error('Function did not throw');
      } catch (e) {
        // Success - function threw
      }
    }
  };
}

// Executar os testes de demonstração
describeTest('CryptoConverter', () => {
  it('should convert from BTC to ETH correctly', async () => {
    // Arrange
    const mockApiClient = new MockCryptoApiClient([
      { symbol: 'BTC', rateUsd: '50000' },
      { symbol: 'ETH', rateUsd: '3000' }
    ]);
    const converter = new CryptoConverter(mockApiClient);
    
    // Act
    const result = await converter.convert(1, 'BTC', 'ETH');
    
    // Assert
    expect(result).toBeCloseTo(50000 / 3000);
  });
  
  it('should throw error for unknown currency', async () => {
    // Arrange
    const mockApiClient = new MockCryptoApiClient();
    const converter = new CryptoConverter(mockApiClient);
    
    // Act & Assert
    try {
      await converter.convert(100, 'USD', 'UNKNOWN');
      console.log('❌ Test failed: Expected error was not thrown');
    } catch (error) {
      expect(error.message).toBe('Currency not found');
    }
  });
});

// Implementação de testes de integração (simulado)
describeTest('Integration Tests', () => {
  it('should interact with multiple components', async () => {
    // Este é um exemplo simplificado de como seria um teste de integração
    // Em um ambiente real, você usaria o framework de teste completo
    
    // Arrange
    const apiClient = new MockCryptoApiClient();
    const converter = new CryptoConverter(apiClient);
    
    // Simular um componente de UI
    class ConverterUI {
      constructor(converter) {
        this.converter = converter;
        this.result = null;
      }
      
      async handleConversion(amount, from, to) {
        try {
          this.result = await this.converter.convert(amount, from, to);
          return { success: true, result: this.result };
        } catch (error) {
          return { success: false, error: error.message };
        }
      }
    }
    
    const ui = new ConverterUI(converter);
    
    // Act
    const response = await ui.handleConversion(2, 'BTC', 'ETH');
    
    // Assert
    expect(response.success).toBe(true);
    expect(response.result).toBeCloseTo(2 * (50000 / 3000));
  });
});

### Estratégias para Aumentar a Testabilidade

1. **Injeção de Dependências** - Permite substituir componentes reais por mocks nos testes
2. **Separação de Responsabilidades** - Facilita o teste de unidades específicas
3. **Princípio da Responsabilidade Única** - Classes com uma única responsabilidade são mais fáceis de testar
4. **Interfaces Claras** - Defina contratos claros entre componentes
5. **Reduzir Efeitos Colaterais** - Funções puras são mais previsíveis e testáveis
6. **Abstrações para APIs Externas** - Crie interfaces para serviços externos
7. **Evitar Estado Global** - Estado global dificulta testes isolados

### Tipos de Testes a Implementar

1. **Testes Unitários** - Testam componentes individuais isoladamente
2. **Testes de Integração** - Testam a interação entre componentes
3. **Testes End-to-End** - Testam o fluxo completo da aplicação
4. **Testes de Performance** - Avaliam o desempenho sob diferentes condições
5. **Testes de Segurança** - Verificam vulnerabilidades de segurança

## Acessibilidade e UX

A acessibilidade é um aspecto essencial para garantir que a aplicação seja utilizável por todas as pessoas, incluindo aquelas com deficiências. Vamos explorar melhorias de acessibilidade e UX.

In [ ]:
<!-- Exemplo de componentes com foco em acessibilidade -->

<!-- Antes: Formulário sem acessibilidade -->
<!--
<div class="converter">
  <div class="field">
    <input type="text" placeholder="Amount">
  </div>
  <div class="field">
    <select>
      <option value="BTC">Bitcoin</option>
      <option value="ETH">Ethereum</option>
    </select>
  </div>
  <div class="field">
    <select>
      <option value="USD">US Dollar</option>
      <option value="EUR">Euro</option>
    </select>
  </div>
  <div class="button">
    <button onclick="convert()">Convert</button>
  </div>
  <div class="result">
    Result: <span id="result">0</span>
  </div>
</div>
-->

<!-- Depois: Formulário com acessibilidade -->
<form class="converter" aria-labelledby="converter-title">
  <h2 id="converter-title">Cryptocurrency Converter</h2>
  
  <div class="field">
    <label for="amount">Amount</label>
    <input 
      type="number" 
      id="amount" 
      name="amount" 
      min="0" 
      step="0.01" 
      required 
      aria-describedby="amount-hint"
    >
    <div id="amount-hint" class="hint">Enter the amount you want to convert</div>
  </div>
  
  <div class="field">
    <label for="from-currency">From Currency</label>
    <select 
      id="from-currency" 
      name="fromCurrency" 
      required
      aria-describedby="from-currency-hint"
    >
      <option value="">Select a currency</option>
      <option value="BTC">Bitcoin (BTC)</option>
      <option value="ETH">Ethereum (ETH)</option>
      <option value="LTC">Litecoin (LTC)</option>
    </select>
    <div id="from-currency-hint" class="hint">Select the source currency</div>
  </div>
  
  <div class="field">
    <label for="to-currency">To Currency</label>
    <select 
      id="to-currency" 
      name="toCurrency" 
      required
      aria-describedby="to-currency-hint"
    >
      <option value="">Select a currency</option>
      <option value="USD">US Dollar (USD)</option>
      <option value="EUR">Euro (EUR)</option>
      <option value="BRL">Brazilian Real (BRL)</option>
    </select>
    <div id="to-currency-hint" class="hint">Select the target currency</div>
  </div>
  
  <div class="actions">
    <button 
      type="submit" 
      aria-label="Convert currencies"
    >
      Convert
    </button>
    <button 
      type="reset" 
      aria-label="Reset form"
    >
      Reset
    </button>
  </div>
  
  <div 
    class="result" 
    aria-live="polite" 
    role="status"
  >
    <h3>Conversion Result</h3>
    <div id="result">Please enter values and submit to see the result</div>
  </div>
  
  <!-- Indicador de carregamento com acessibilidade -->
  <div 
    id="loading-indicator" 
    class="loading-indicator" 
    role="progressbar" 
    aria-hidden="true"
  >
    Loading...
  </div>
  
  <!-- Mensagens de erro com acessibilidade -->
  <div 
    id="error-message" 
    class="error-message" 
    role="alert" 
    aria-hidden="true"
  ></div>
</form>

In [ ]:
/* Estilos focados em acessibilidade */

/* Variáveis para consistência */
:root {
  --primary-color: #3498db;
  --error-color: #e74c3c;
  --success-color: #2ecc71;
  --text-color: #333;
  --background-color: #fff;
  --focus-color: #2980b9;
  --border-color: #ddd;
  --border-radius: 4px;
  --spacing-unit: 16px;
  --font-size-base: 16px;
  --font-size-small: 14px;
  --font-size-large: 18px;
}

/* Reset básico */
* {
  box-sizing: border-box;
  margin: 0;
  padding: 0;
}

body {
  font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
  line-height: 1.5;
  color: var(--text-color);
  background-color: var(--background-color);
  font-size: var(--font-size-base);
}

/* Estilos do formulário */
.converter {
  max-width: 600px;
  margin: var(--spacing-unit) auto;
  padding: var(--spacing-unit);
  border: 1px solid var(--border-color);
  border-radius: var(--border-radius);
  background-color: #f9f9f9;
}

.field {
  margin-bottom: var(--spacing-unit);
}

/* Estilos para labels acessíveis */
label {
  display: block;
  margin-bottom: calc(var(--spacing-unit) / 2);
  font-weight: bold;
}

/* Alto contraste para melhor legibilidade */
input, select, button {
  width: 100%;
  padding: calc(var(--spacing-unit) / 2);
  border: 2px solid var(--border-color);
  border-radius: var(--border-radius);
  font-size: var(--font-size-base);
  color: var(--text-color);
}

/* Estado de foco bem visível */
input:focus, select:focus, button:focus {
  outline: 3px solid var(--focus-color);
  outline-offset: 2px;
  border-color: var(--primary-color);
}

/* Estilos para estado de erro */
input.error, select.error {
  border-color: var(--error-color);
}

/* Dicas para usuários */
.hint {
  font-size: var(--font-size-small);
  color: #666;
  margin-top: 4px;
}

/* Botões acessíveis */
button {
  background-color: var(--primary-color);
  color: white;
  border: none;
  padding: calc(var(--spacing-unit) / 2);
  cursor: pointer;
  font-weight: bold;
  transition: background-color 0.3s;
}

button:hover {
  background-color: var(--focus-color);
}

button:active {
  transform: translateY(1px);
}

button[type="reset"] {
  background-color: #95a5a6;
}

.actions {
  display: flex;
  gap: calc(var(--spacing-unit) / 2);
  margin-bottom: var(--spacing-unit);
}

/* Resultado com realce visual */
.result {
  margin-top: var(--spacing-unit);
  padding: var(--spacing-unit);
  background-color: #f0f0f0;
  border-radius: var(--border-radius);
}

/* Indicador de carregamento */
.loading-indicator {
  display: none;
  text-align: center;
  padding: calc(var(--spacing-unit) / 2);
  color: var(--primary-color);
}

.loading-indicator[aria-hidden="false"] {
  display: block;
}

/* Mensagens de erro */
.error-message {
  display: none;
  color: var(--error-color);
  background-color: rgba(231, 76, 60, 0.1);
  border: 1px solid var(--error-color);
  padding: calc(var(--spacing-unit) / 2);
  margin-top: var(--spacing-unit);
  border-radius: var(--border-radius);
}

.error-message[aria-hidden="false"] {
  display: block;
}

/* Responsividade */
@media (max-width: 768px) {
  .converter {
    width: 95%;
    padding: calc(var(--spacing-unit) / 2);
  }
  
  .actions {
    flex-direction: column;
  }
  
  button {
    margin-bottom: calc(var(--spacing-unit) / 2);
  }
}

/* Suporte para modo escuro */
@media (prefers-color-scheme: dark) {
  :root {
    --text-color: #f0f0f0;
    --background-color: #222;
    --border-color: #444;
    --primary-color: #4d90fe;
    --focus-color: #6ba4ff;
  }
  
  .converter {
    background-color: #333;
  }
  
  .result {
    background-color: #444;
  }
  
  .hint {
    color: #aaa;
  }
}

/* Suporte para preferências de movimento reduzido */
@media (prefers-reduced-motion: reduce) {
  button, input, select {
    transition: none;
  }
  
  button:active {
    transform: none;
  }
}